In [10]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from matplotlib.dates import  DateFormatter
import datetime
from datetime import timedelta
import time
import pickle
import seaborn as sns

import astropy
import astropy.constants as const
import astropy.units as u
#import astroquery  
import sunpy
from sunpy.coordinates import frames, get_horizons_coord
from sunpy.time import parse_time
import astrospice

#import heliopy.data.spice as spicedata
#import heliopy.spice as spice



import sys
import os
#import urllib
#import json
#import importlib
import pandas as pd
#import copy
#import openpyxl
#import h5py
import cdflib


import warnings
warnings.filterwarnings('ignore')


#import heliosat

#on mac
#sys.path.append('/Users/chris/python/heliocats')

#from heliocats import plot as hp
#importlib.reload(hp) #reload again while debugging

#from heliocats import data as hd
#importlib.reload(hd) #reload again while debugging

#from heliocats import cats as hc
#importlib.reload(hc) #reload again while debugging

#from heliocats import stats as hs
#importlib.reload(hs) #reload again while debugging

#where the in situ data files are located is read 
#from config.py 

#import config
#importlib.reload(config)
#from config import data_path
#from config import data_path_ML


plt.rcParams["figure.figsize"] = (15,5)


from astropy.constants import au
AU=au.value/1e3 #in km

from astropy.constants import R_sun
RS=R_sun.value*1e-3 #in km

print('done')

done


In [11]:
ll_path = "py3dcore_h4c/custom_data/solo_swa_2022sep"

files = os.listdir(ll_path)
files.sort()
llfiles = [os.path.join(ll_path, f) for f in files]
#print(llfiles)

timep=np.zeros(0,dtype=[('time',object)])
den=np.zeros(0)
temp=np.zeros(0)
vr=np.zeros(0)
vt=np.zeros(0)
vn=np.zeros(0)


for i in np.arange(0,len(llfiles)):
    p1 = cdflib.CDF(llfiles[i])
    #p1.cdf_info()

    den1=p1.varget('N')
    speed1=p1.varget('V_RTN')
    temp1=p1.varget('T')

    vr1=speed1[:,0]
    vt1=speed1[:,1]
    vn1=speed1[:,2]
    
    vr=np.append(vr1,vr)
    vt=np.append(vt1,vt)
    vn=np.append(vn1,vn)
    
    
    temp=np.append(temp1,temp)
    den=np.append(den1,den)

    
    time1=p1.varget('EPOCH')
    t1=parse_time(cdflib.cdfastropy.convert_to_astropy(time1, format=None)).datetime
    timep=np.append(timep,t1)
    #print(time1)


temp=temp*(1.602176634*1e-19)/(1.38064852*1e-23) # from ev to K 

ll_path = "py3dcore_h4c/custom_data/solo_mag_2022sep"
#ll_path = "/nas/helio/data/SolarOrbiter/MAG/low_latency/"

files = os.listdir(ll_path)
files.sort()
llfiles = [os.path.join(ll_path, f) for f in files]
print(llfiles)

br1=np.zeros(0)
bt1=np.zeros(0)
bn1=np.zeros(0)
time1=np.zeros(0,dtype=[('time',object)])


for i in np.arange(0,len(llfiles)):
    m1 = cdflib.CDF(llfiles[i])
    #print(m1.cdf_info())


    b=m1.varget('B_RTN')
    br=b[:,0]
    bt=b[:,1]
    bn=b[:,2]
    
    br1=np.append(br1,br)
    bt1=np.append(bt1,bt)
    bn1=np.append(bn1,bn)

    time=m1.varget('EPOCH')

    t1=parse_time(cdflib.cdfastropy.convert_to_astropy(time, format=None)).datetime
    time1=np.append(time1,t1)

        
print(m1.attget('UNITS','B_RTN')) 


    
#define datetime object for 1 minute resolution    
starttime = datetime.datetime(2022, 9, 1)
endtime = datetime.datetime(2022, 9,12)
time_int = []
while starttime < endtime:
        time_int.append(starttime)
        starttime += timedelta(minutes=1)

        
time_int_mat=mdates.date2num(time_int)
time1_mat=mdates.date2num(time1)
timep_mat=mdates.date2num(timep)        

solo_ll=np.zeros(np.size(time_int),dtype=[('time',object),('bx', float),('by', float),\
            ('bz', float),('bt', float),('r', float),('lat', float),('lon', float),\
            ('x', float),('y', float),('z', float),('vx', float),\
            ('vy', float),('vz', float),('vt', float),('tp', float),('np', float) ] )   

solo_ll = solo_ll.view(np.recarray)  


solo_ll.time=time_int
solo_ll.bx=np.interp(time_int_mat, time1_mat,br1)
solo_ll.by=np.interp(time_int_mat, time1_mat,bt1)
solo_ll.bz=np.interp(time_int_mat, time1_mat,bn1)
solo_ll.bt=np.sqrt(solo_ll.bx**2+solo_ll.by**2+solo_ll.bz**2)


solo_ll.np=np.interp(time_int_mat, timep_mat,den)
solo_ll.tp=np.interp(time_int_mat, timep_mat,temp) 
solo_ll.vx=np.interp(time_int_mat, timep_mat,vr)
solo_ll.vy=np.interp(time_int_mat, timep_mat,vt)
solo_ll.vz=np.interp(time_int_mat, timep_mat,vn)
solo_ll.vt=np.sqrt(solo_ll.vx**2+solo_ll.vy**2+solo_ll.vz**2)


#spacecraft position with astrospice
kernels = astrospice.registry.get_kernels('solar orbiter', 'predict')
solo_kernel = kernels[0]

solo_coords = astrospice.generate_coords('solar orbiter', time_int)
solo_coords_heeq = solo_coords.transform_to(sunpy.coordinates.HeliographicStonyhurst())

solo_ll.lon=solo_coords_heeq.lon.value
solo_ll.lat=solo_coords_heeq.lat.value
solo_ll.r=solo_coords_heeq.radius.to(u.au).value

['py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220831T000215-20220901T000214_V02C.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220901T000215-20220902T000214_V02C.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220902T000215-20220903T000215_V04I.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220903T000216-20220904T000215_V04I.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220904T000216-20220905T000215_V04I.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220905T000216-20220906T000215_V04I.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220906T000216-20220907T000215_V03C.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220907T000216-20220908T000215_V03C.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220908T000216-20220909T000216_V02C.cdf', 'py3dcore_h4c/custom_data/solo_mag_2022sep/solo_LL02_mag_20220909T000217-20220910T000216_V03C.cdf',

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

solo_ANC_soc-orbit_20200210-20301120_L016_V2_00025_V01.bsp:   0%|          | 0.00/3.46M [00:00<?, ?B/s]

In [15]:
time1[-1]

datetime.datetime(2022, 9, 13, 0, 3, 20, 673560)

In [1]:
import pickle
import pandas as pds
import datetime

%load_ext autoreload

%autoreload 2


In [3]:
import os
import cdflib
from sunpy.time import parse_time
from datetime import timedelta
import matplotlib.dates as mdates
import numpy as np

#locally on Macbook
ll_path = "py3dcore_h4c/custom_data/solo_mag_2022sep"
#ll_path = "/nas/helio/data/SolarOrbiter/MAG/low_latency/"

files = os.listdir(ll_path)
files.sort()
llfiles = [os.path.join(ll_path, f) for f in files]
#print(llfiles)

timep=np.zeros(0,dtype=[('time',object)])
den=np.zeros(0)
temp=np.zeros(0)
vr=np.zeros(0)
vt=np.zeros(0)
vn=np.zeros(0)


br1=np.zeros(0)
bt1=np.zeros(0)
bn1=np.zeros(0)
time1=np.zeros(0,dtype=[('time',object)])

for i in np.arange(0,len(llfiles)):
    m1 = cdflib.CDF(llfiles[i])
    #print(m1.cdf_info())


    b=m1.varget('B_RTN')
    br=b[:,0]
    bt=b[:,1]
    bn=b[:,2]
    
    br1=np.append(br1,br)
    bt1=np.append(bt1,bt)
    bn1=np.append(bn1,bn)

    time=m1.varget('EPOCH')

    t1=parse_time(cdflib.cdfastropy.convert_to_astropy(time, format=None)).datetime
    time1=np.append(time1,t1)
    
#define datetime object for 1 minute resolution    
starttime = datetime.datetime(2022, 9, 1)
endtime = datetime.datetime(2022, 9,12)
time_int = []
while starttime < endtime:
        time_int.append(starttime)
        starttime += timedelta(minutes=1)

        
time_int_mat=mdates.date2num(time_int)
time1_mat=mdates.date2num(time1)
timep_mat=mdates.date2num(timep)        

solo_ll=np.zeros(np.size(time_int),dtype=[('time',object),('bx', float),('by', float),\
            ('bz', float),('bt', float),('r', float),('lat', float),('lon', float),\
            ('x', float),('y', float),('z', float),('vx', float),\
            ('vy', float),('vz', float),('vt', float),('tp', float),('np', float) ] )   

solo_ll = solo_ll.view(np.recarray)  


solo_ll.time=time_int
solo_ll.bx=np.interp(time_int_mat, time1_mat,br1)
solo_ll.by=np.interp(time_int_mat, time1_mat,bt1)
solo_ll.bz=np.interp(time_int_mat, time1_mat,bn1)
solo_ll.bt=np.sqrt(solo_ll.bx**2+solo_ll.by**2+solo_ll.bz**2)

print(solo_ll)
solo_ll.np=np.interp(time_int_mat, timep_mat,den)
solo_ll.tp=np.interp(time_int_mat, timep_mat,temp) 
solo_ll.vx=np.interp(time_int_mat, timep_mat,vr)
solo_ll.vy=np.interp(time_int_mat, timep_mat,vt)
solo_ll.vz=np.interp(time_int_mat, timep_mat,vn)
solo_ll.vt=np.sqrt(solo_ll.vx**2+solo_ll.vy**2+solo_ll.vz**2)


#spacecraft position with astrospice
kernels = astrospice.registry.get_kernels('solar orbiter', 'predict')
solo_kernel = kernels[0]

solo_coords = astrospice.generate_coords('solar orbiter', time_int)
solo_coords_heeq = solo_coords.transform_to(sunpy.coordinates.HeliographicStonyhurst())

solo_ll.lon=solo_coords_heeq.lon.value
solo_ll.lat=solo_coords_heeq.lat.value
solo_ll.r=solo_coords_heeq.radius.to(u.au).value

[(datetime.datetime(2022, 9, 1, 0, 0), 48.88674775,  -0.93824528, -12.11082891, 50.37327255, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)
 (datetime.datetime(2022, 9, 1, 0, 1), 51.42453906,  -2.40044487, -15.52426593, 53.77032812, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)
 (datetime.datetime(2022, 9, 1, 0, 2), 51.52859063,  -2.06661932, -15.49528578, 53.84765965, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)
 ...
 (datetime.datetime(2022, 9, 11, 23, 57),  3.06703965, -18.05248273,  -1.75860013, 18.39542169, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)
 (datetime.datetime(2022, 9, 11, 23, 58),  2.92522907, -18.10355732,  -1.54155706, 18.40304733, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)
 (datetime.datetime(2022, 9, 11, 23, 59),  2.90189963, -18.49883675,  -1.65545716, 18.79809887, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.)]


TypeError: Cannot cast array data from dtype([('time', 'O')]) to dtype('float64') according to the rule 'safe'

In [56]:
time1

array([datetime.datetime(2022, 8, 31, 0, 3, 32, 301678),
       datetime.datetime(2022, 8, 31, 0, 3, 40, 301692),
       datetime.datetime(2022, 8, 31, 0, 3, 48, 301706), ...,
       datetime.datetime(2022, 9, 13, 0, 3, 4, 673532),
       datetime.datetime(2022, 9, 13, 0, 3, 12, 673546),
       datetime.datetime(2022, 9, 13, 0, 3, 20, 673560)], dtype=object)

In [5]:
import heliosat
from py3dcore_h4c.fitter.base import custom_observer

observer_obj = custom_observer('solo_2022_sep_mag_ll_swa_science.p')

In [31]:
file = pickle.load(open('py3dcore_h4c/custom_data/solo_2022_sep_mag_ll_swa_science.p', 'rb'))
data = pds.DataFrame(file)
#data['time'] = pds.to_datetime(data['time'])
#data.set_index('time',  inplace=True)
#data.index.name = None
#data.index = data.index.tz_localize(None)

In [32]:
import numpy as np

file['x'] = file['r'] * np.sin(file['lat']) * np.cos(file['lon'])

In [33]:
file['x']

array([ 0.13914297,  0.13910127,  0.13905956, ..., -0.05783126,
       -0.05811244, -0.05839362])

In [34]:
import py3dcore_h4c

fitter = py3dcore_h4c.ABC_SMC()

model_kwargs = {
        "ensemble_size": int(2**17), #2**17
        "iparams": {
           "cme_longitude": {
               "maximum": -110,
               "minimum": -130
           },
           "cme_latitude": {
               "maximum": -10,
               "minimum": -50
           },
           "cme_inclination": {
               "maximum": 10,
               "minimum": 0
           }, 
            "cme_launch_velocity": {
                "maximum": 750,
                "minimum": 250
            },
            "cme_launch_radius": {
                "maximum": 30,
                "minimum": 5
            }
        }
    }

t_launch = datetime.datetime(2022, 6, 2, 6, tzinfo=datetime.timezone.utc)

t_s_psp = datetime.datetime(2022, 6, 2, 12, tzinfo=datetime.timezone.utc)
t_e_psp = datetime.datetime(2022, 6, 3, 6, tzinfo=datetime.timezone.utc)

t_psp = [datetime.datetime(2022, 6, 2, 13, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 15, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc)]


fitter.initialize(t_launch, py3dcore_h4c.ToroidalModel, model_kwargs)
fitter.add_observer("PSP", t_psp, t_s_psp, t_e_psp)



In [35]:
fitter.observers

[['PSP',
  [datetime.datetime(2022, 6, 2, 13, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 14, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 15, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 16, 0, tzinfo=datetime.timezone.utc)],
  datetime.datetime(2022, 6, 2, 12, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2022, 6, 3, 6, 0, tzinfo=datetime.timezone.utc),
  None]]

In [36]:
import heliosat

observer_obj = getattr(heliosat, 'PSP')()

In [37]:
dt_all = [t_s_psp] + t_psp + [t_e_psp]

_, data = observer_obj.get([t_s_psp, t_e_psp], "mag", reference_frame= 'HEEQ', sampling_freq=300, use_cache=True, as_endpoints=True)
tra = observer_obj.trajectory(dt_all, reference_frame='HEEQ', data_key = "pos")

In [38]:
tra

array([[ 0.03270647, -0.06253744, -0.00435444],
       [ 0.03625155, -0.06203463, -0.00433431],
       [ 0.03974751, -0.06145127, -0.00430821],
       [ 0.04319256, -0.0607926 , -0.00427648],
       [ 0.04658539, -0.06006374, -0.00423949],
       [ 0.08857316, -0.04483293, -0.00332971]])

In [17]:
from py3dcore_h4c.fitter.base import FittingData

data_obj = FittingData(fitter.observers, 'HEEQ') 
data_obj.generate_noise("psd", 300)

[[-1.05285896e+02 -5.62792175e+02 -2.43961960e+02]
 [ 7.42871170e+01 -6.29402954e+02 -7.24466019e+01]
 [-6.05077019e+01 -6.28743042e+02 -1.68775635e+02]
 [-7.53965855e+00 -6.56512024e+02 -3.57362442e+01]
 [-3.54195023e+01 -6.26810242e+02  1.90341187e+02]
 [-6.03448601e+01 -6.51321289e+02 -6.79928055e+01]
 [-3.37802315e+01 -7.07948975e+02 -9.59103546e+01]
 [-9.28271027e+01 -7.36282532e+02 -9.82228165e+01]
 [-7.60810242e+01 -7.27990356e+02 -2.86131039e+01]
 [-9.31273270e+01 -4.55535217e+02 -2.68363068e+02]
 [-1.21022545e+02 -5.58738342e+02  1.40962267e+01]
 [-1.12149986e+02 -5.89495667e+02 -5.60987663e+01]
 [-8.21963806e+01 -5.91795898e+02  2.48691463e+01]
 [ 6.68744736e+01 -6.42287598e+02  1.37235550e+02]
 [ 3.46275787e+01 -6.24822937e+02  1.33888809e+02]
 [ 8.53221512e+01 -6.16764648e+02  1.87400177e+02]
 [ 1.20594688e+02 -5.99951904e+02  1.82320099e+02]
 [ 1.68908127e+02 -5.67987976e+02  1.68295822e+02]
 [ 2.26623611e+02 -5.95109497e+02  1.27217896e+02]
 [ 2.61118286e+02 -5.60616089e+

In [26]:
from py3dcore_h4c.fitter.base import CustomData

data_obj = CustomData(fitter.observers, 'HEEQ','solo_2022_sep_mag_ll_swa_science.p') 

In [35]:
import numpy as np

_ = np.linspace(t_s_psp.timestamp(), t_e_psp.timestamp(), int((t_e_psp.timestamp() - t_s_psp.timestamp()) // 300))  
dtp = [datetime.datetime.fromtimestamp(_, datetime.timezone.utc) for _ in _]
tt = [x.replace(tzinfo=None) for x in dtp]

In [25]:
,second=0, microsecond=0 [0]

[datetime.datetime(2022, 9, 1, 0, 0),
 datetime.datetime(2022, 9, 1, 0, 5),
 datetime.datetime(2022, 9, 1, 0, 10),
 datetime.datetime(2022, 9, 1, 0, 15),
 datetime.datetime(2022, 9, 1, 0, 20),
 datetime.datetime(2022, 9, 1, 0, 25),
 datetime.datetime(2022, 9, 1, 0, 30),
 datetime.datetime(2022, 9, 1, 0, 35),
 datetime.datetime(2022, 9, 1, 0, 40),
 datetime.datetime(2022, 9, 1, 0, 45),
 datetime.datetime(2022, 9, 1, 0, 50),
 datetime.datetime(2022, 9, 1, 0, 55),
 datetime.datetime(2022, 9, 1, 1, 0),
 datetime.datetime(2022, 9, 1, 1, 5),
 datetime.datetime(2022, 9, 1, 1, 10),
 datetime.datetime(2022, 9, 1, 1, 15),
 datetime.datetime(2022, 9, 1, 1, 20),
 datetime.datetime(2022, 9, 1, 1, 25),
 datetime.datetime(2022, 9, 1, 1, 30),
 datetime.datetime(2022, 9, 1, 1, 35),
 datetime.datetime(2022, 9, 1, 1, 40),
 datetime.datetime(2022, 9, 1, 1, 45),
 datetime.datetime(2022, 9, 1, 1, 50),
 datetime.datetime(2022, 9, 1, 1, 55),
 datetime.datetime(2022, 9, 1, 2, 0),
 datetime.datetime(2022, 9, 1,

In [42]:
ii = [np.where(file['time']==x)[0][0] for x in tt if np.where(file['time']==x)[0].size > 0]

In [43]:
ii

[0, 576, 1152, 1728, 2304, 2880]

In [27]:
data_obj.generate_noise("psd", 300)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/hruedisser/miniconda3/envs/newtest/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_25285/3518075560.py", line 1, in <module>
    data_obj.generate_noise("psd", 300)
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 588, in generate_noise
    data = observer_obj.get([dt_s, dt_e], "mag", reference_frame=self.reference_frame, sampling_freq=sampling_freq, use_cache=True, as_endpoints=True)
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 411, in get
    ii = [np.where(self.data['time']==x)[0][0] for x in tt]
  File "/home/hruedisser/py3DCORE_h4c/py3dcore_h4c/fitter/base.py", line 411, in <listcomp>
    ii = [np.where(self.data['time']==x)[0][0] for x in tt]
IndexError: index 0 is out of bounds for axis 0 with size 0

During handling of the above exception, another exception occurre

In [12]:
data_obj.generate_data([0])

In [9]:
from py3dcore_h4c.fitter.base import FittingData

data_obj = FittingData(fitter.observers, 'HEEQ') 

In [74]:
fitter.observers

[['PSP',
  [datetime.datetime(2022, 6, 2, 13, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 14, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 15, 0, tzinfo=datetime.timezone.utc),
   datetime.datetime(2022, 6, 2, 16, 0, tzinfo=datetime.timezone.utc)],
  datetime.datetime(2022, 6, 2, 12, 0, tzinfo=datetime.timezone.utc),
  datetime.datetime(2022, 6, 3, 6, 0, tzinfo=datetime.timezone.utc),
  None]]

In [50]:
t_psp = [
        datetime.datetime(2022, 9, 11, 0, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc) ,
        datetime.datetime(2022, 6, 2, 15, tzinfo=datetime.timezone.utc),
        datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc),
    ] 

In [87]:
data.loc[t_psp]

KeyError: "None of [DatetimeIndex(['2022-06-02 13:00:00+00:00', '2022-06-02 14:00:00+00:00',\n               '2022-06-02 15:00:00+00:00', '2022-06-02 16:00:00+00:00'],\n              dtype='datetime64[ns, UTC]', freq=None)] are in the [index]"

In [52]:
datetime.datetime.timestamp(t_psp[0])

1662854400.0

In [37]:
data[t_psp]

KeyError: "None of [DatetimeIndex(['2022-09-11 00:00:00+00:00', '2022-06-02 14:00:00+00:00',\n               '2022-06-02 15:00:00+00:00', '2022-06-02 16:00:00+00:00'],\n              dtype='datetime64[ns, UTC]', freq=None)] are in the [columns]"

In [38]:
data.index[0]

Timestamp('2022-09-01 00:00:00')